In [1]:
import time
import requests
import numba as nb

from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from random import randrange
from scipy.special import comb

In [2]:
url='https://www.janestreet.com/puzzles/robot-swimming-trials-index/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
y =[text for text in soup.body.stripped_strings]

#display([(i,j) for i,j in enumerate(y)])
display(Markdown("### "+y[7]+"\n\n"+str("\n".join(y[9:91]))))

### Robot Swimming Trials

Show Solution
As we have
twice
before
, we find ourselves engrossed in a robot sports competition. In the Robot Swimming Trials, 3
N
identical robots compete for
N
equivalent spots in the finals by swimming
N
races. Each robot precommits to spending a certain amount of its fuel in each race. After all the races are run, the spots in the finals are given to the winners of the races, moving from the fastest winner to the slowest. (Once a robot wins a race, it is ineligible to win another race.) A robot’s speed is strictly increasing in the amount of fuel it spends, and ties are broken by randomly choosing the winner among the robots that have spent the same amount of fuel.
Mathematically speaking, the 3
N
robots each submit a strategy, which is an
N
-tuple of nonnegative real number “bids” summing to 1, representing the fuel burned in each of the
N
races. The winners of the races are then determined from the highest bid (across all races and all robots) on down, with ties broken randomly. Once a robot wins a race their other bids are deleted, so we are guaranteed to get
N
distinct qualifiers for the finals.
For example, suppose
N
=3 and the 3
N
=9 robots submit their strategies as
Robot
Race 1
Race 2
Race 3
Automatonya
0.6
0.1
0.3
Botty
0.6
0.3
0.1
Chroma
0
1
0
Data
0.3
0.5
0.2
Electro
0.2
0.8
0
Fernandroid
0.4
0.5
0.1
Gregulator
0.5
0.5
0
Hannanobot
0
0.9
0.1
IO
0.2
0.7
0.1
The second race gets resolved first because Chroma’s bid of 1 is the highest overall, and Chroma is declared the winner of that time trial. Next, the first race is resolved because 0.6 is the highest remaining bid (we ignore the 0.9, 0.8, and 0.7 in the second race because it already has a winner). We flip a fair coin to determine who is the winner between Automatonya and Botty; say that Automatonya gets lucky and is declared the winner. Then the third race is decided, and Data is declared the winner, because 0.2 is the highest bid among robots that have not yet won (Automatonya’s 0.3 is ignored).
Over the storied history of the RST, the metagame settled into what was widely believed to be the Nash equilibrium: each robot uniformly randomly selects a race and devotes all of their fuel to it. Let’s call this the
discrete strategy
. However, rumors are circulating that this conventional wisdom is not entirely accurate: for a large enough
N
, the discrete strategy is not the Nash equilibrium. You’ve been tasked to find two pieces of information:
What is the smallest
N
for which the trial does
not
have the discrete strategy as the Nash equilibrium?
For this
N
, if the other 3
N
-1 robots naively play the discrete
strategy and your robot plays optimally (exploiting this knowledge of
your opponents’ strategies), with what probability
p

In [3]:
# A some point there is a larger than 1/3 probability of there being at least one race where every other robot choses zero. 
# So if the last robot spreads evenly over the other races he will win any race where there are no other robots who choose 1.
# The probability of throwing N balls into k bins and there being a ball in every bin is as per the link below.
# https://math.stackexchange.com/questions/174674/if-n-balls-are-thrown-into-k-bins-what-is-the-probability-that-every-bin-gets-a

$P(X=0)=\sum_{j=0}^k (-1)^j {k\choose j}\left(1-{j\over k}\right)^n$

In [8]:
def calc(k):
    n = 3* k - 1
    return 1-sum([pow(-1,h)*comb(k,h)*(1-h/k)**n for h in range(k)])

for i in range(2,10):
    n=i*3
    print("For {} rounds the chance is {:.6f} ".format(i,calc(i)))

For 2 rounds the chance is 0.062500 
For 3 rounds the chance is 0.116598 
For 4 rounds the chance is 0.166012 
For 5 rounds the chance is 0.212093 
For 6 rounds the chance is 0.255368 
For 7 rounds the chance is 0.296128 
For 8 rounds the chance is 0.334578 
For 9 rounds the chance is 0.370878 


In [5]:
url='https://www.janestreet.com/puzzles/robot-swimming-trials-solution/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
y =[text for text in soup.body.stripped_strings]
#display([(i,j) for i,j in enumerate(y)])
display(Markdown("### "+y[8]+"\n\n"+str("\n".join(y[10:122]))))

### October 2021 : Solution

We are given that our opponents are all playing the discrete strategy,
which means they will all be independently choosing a race uniformly
randomly and devoting
all of their fuel to it. Since we are searching for the smallest
N
so that the discrete strategy is not optimal, we will be playing
something other than this strategy. This means we won’t be assigning
all of our fuel to any one race, but since all of our opponents are
“bidding” 1 or 0 on every race, the best non-discrete strategies will
involve bidding a nonzero amount on every race, which will beat all of
the 0 bids (but lose to all of the 1 bids). So without loss of
generality we can assume the strategy of bidding 1/
N
on every race,
we can call this the
uniform
strategy.
Now we need to compute the smallest
N
for which this beats out the
discrete strategy. The uniform strategy wins a race exactly when none
of the 3
N
-1 other discrete-strategy-playing robots select that race
for their fuel. It’s straightforward to compute this probability for a
single race, (1-1/
N
)^(3
N
-1). But the assignment of discrete
strategies is
not independent
across the
N
races! Assuming they were
would give an incorrect answer of
N
=9 and
p
=0.350245…
Also, these events of winning a race with the uniform strategy are
not disjoint
across the
N
races! Assuming they were would give
an incorrect answer of
N
=8 and
p
=0.370916…
Instead, a nice recursion could be found to count up the number of
arrangements of the 3
N
-1 other players that left at least one race
undefended. For example, let P(
R
,
m
,
n
) equal the probability that, if we
need to assign
R
robots to (
m
+
n
) total races,
m
of which already have
a robot assigned and
n
of which don’t, we will eventually assign at
least one robot to all (
m
+
n
) races. Then assigning the next robot to a
race uniformly randomly implies the recursion:
P(
R
,
m
,
n
) = (
m
* P(
R
-1,
m
,
n
) +
n
* P(
R
-1,
m
+1,
n
-1))/(
m
+
n
).
Along with the boundary values
P(R,
m
,0) = 1 (all races have been assigned at least one robot)
and for
n
>0,
P(0,
m
,
n
) = 0 (we’ve run out of robots and we still have an unassigned
race),
we can compute values of P efficiently by induction. We want to find
the smallest
N
such that
1 - P(3
N
-1,0,
N
) > 1/3,
which occurs at
N
=
8
and
p
=
0.334578…
.

In [6]:
# Check with some simulations
@nb.njit()
def sim(N):
    picks = [randrange(1, N+1) for p in range(N*3-1)]
    for n in range(1,N+1):
        if n not in picks:
            return 1
    return 0

@nb.njit()
def run_it(N,sims):
    tot = 0
    for x in range(sims):
        tot += sim(N)
    return tot/sims


In [7]:
start= time.time()
print("Rounds\tSim\t\tCalc")
for N in range(2,10):
    sims = 10000000
    x = run_it(N,sims)
    print("{:>3}\t{:.6f}\t{:.6f}".format(N,x,calc(N)))
print("took",time.time()-start)

Rounds	Sim		Calc
  2	0.062389	0.062500
  3	0.116644	0.116598
  4	0.166192	0.166012
  5	0.212212	0.212093
  6	0.255415	0.255368
  7	0.296339	0.296128
  8	0.334660	0.334578
  9	0.370837	0.370878
took 44.56086087226868
